In [1]:
!pip install -U spacy
!pip install scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bc5cdr_md-0.4.0.tar.gz
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bionlp13cg_md-0.4.0.tar.gz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 27.1 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.5.2
    Uninstalling spacy-3.5.2:
      Successfully uninstalled spacy-3.5.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 18.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 9.7 MB/s eta 0:00:00
  Using cached pybind11-2.6.1-py2.py3-none-any.whl (188 kB)
  Created wheel for nmslib: filename=nmslib-2.1.1-cp310-cp310-linux_x86_64.whl size=13572390 sha256=1e3b6c9dff1abc63385029c7ca2c13df994861c8d7de2d652d977a

In [3]:
import pandas as pd
import re
import numpy as np
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import scispacy
import spacy
import en_ner_bc5cdr_md
import en_ner_bionlp13cg_md
from spacy import displacy

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
data=pd.read_csv("/content/vet.csv")
df1=pd.read_csv("/content/vetabb.csv")
df1['abb'] = df1['abb'].str.upper()
df1['simple_chemical']=df1['simple_chemical'].str.capitalize()
df1['chemical']=df1['chemical'].str.capitalize()
data['Consult_date'] = pd.to_datetime(data['Consult_date'])
data['year'] = data['Consult_date'].dt.year
data['month'] = data['Consult_date'].dt.strftime('%B')

In [6]:
data.isnull().sum()

Unnamed: 0             0
SAVSNET_consult_id     0
Narrative              0
SAVSNET MPC           15
Consult_date           0
Species                0
year                   0
month                  0
dtype: int64

In [7]:
data.shape

(4415, 8)

In [8]:
data['Narrative'] = np.where(data['Narrative'] =='""', None, data['Narrative'])
data.isnull().sum()

Unnamed: 0              0
SAVSNET_consult_id      0
Narrative             178
SAVSNET MPC            15
Consult_date            0
Species                 0
year                    0
month                   0
dtype: int64

In [9]:
data = data.dropna(subset=['Narrative'])
data.shape

(4237, 8)

In [10]:
data=data.sort_values(by="Narrative", key=lambda x: x.str.len())
data

,Unnamed: 0,SAVSNET_consult_id,Narrative,SAVSNET MPC,Consult_date,Species,year,month
218,218,2200513,""".""",vaccination,2016-09-22 15:06:00,cat,2016,September
4386,4386,330871,""".""",other_unwell,2015-02-02 12:11:00,dog,2015,February
1197,1197,5452601,38.2,other_unwell,2018-07-12 15:16:00,cat,2018,July
2655,2655,4880367,"""-.""",other_healthy,2018-10-08 10:33:00,dog,2018,October
2595,2595,3223610,"""-.""",other_healthy,2017-08-06 09:51:00,dog,2017,August
...,...,...,...,...,...,...,...,...
1674,1674,6443213,"CHECK EYES AND OFF COLOUR. See history, tear ...",other_unwell,2019-07-26 15:10:00,dog,2019,July
3138,3138,6563147,4.5kg BCS 4/9 - Lethargy/inappetence. O report...,other_unwell,2019-08-19 18:44:00,cat,2019,August
2161,2161,5404875,Presented for Annual HC and Booster. No health...,vaccination,2018-11-27 16:43:00,dog,2018,November
768,768,5492258,LUMBAR PAIN and LH weakness. <<identifier>> wa...,other_healthy,2018-12-17 13:55:00,dog,2018,December


In [11]:
data=data.loc[data.index.drop([218, 4386, 2595,1197,2655,3087,4264,2353,413,4125,4221,557,4031,3635,2508,3643,174,2364,1425,4198,3453,939,975,2544,606,161,2826,1647,2374,1546,1324,1643,2730,4389,43,804,3103,2567,1148,2428,2416,3064,3107,2250,2116])]
data

,Unnamed: 0,SAVSNET_consult_id,Narrative,SAVSNET MPC,Consult_date,Species,year,month
3619,3619,6541908,istat,other_healthy,2019-08-15 09:07:00,cat,2019,August
3417,3417,6809034,2nd l4,vaccination,2019-10-14 17:35:00,dog,2019,October
4328,4328,6765717,vomiting,gastroenteric,2019-04-10 09:52:00,dog,2019,April
523,523,460793,"""L4 booster""",vaccination,2015-01-07 09:36:00,dog,2015,January
1480,1480,5566996,Rp Protexin.,other_healthy,2019-08-01 17:14:00,dog,2019,August
...,...,...,...,...,...,...,...,...
1674,1674,6443213,"CHECK EYES AND OFF COLOUR. See history, tear ...",other_unwell,2019-07-26 15:10:00,dog,2019,July
3138,3138,6563147,4.5kg BCS 4/9 - Lethargy/inappetence. O report...,other_unwell,2019-08-19 18:44:00,cat,2019,August
2161,2161,5404875,Presented for Annual HC and Booster. No health...,vaccination,2018-11-27 16:43:00,dog,2018,November
768,768,5492258,LUMBAR PAIN and LH weakness. <<identifier>> wa...,other_healthy,2018-12-17 13:55:00,dog,2018,December


In [12]:
data.shape

(4192, 8)

In [13]:
text=[]
for i in data['Narrative']:
  text.append(re.sub(r'\+', ' and ', i))
data['text']=text

In [14]:
for index, row in data.iterrows():
    dt=word_tokenize(row["text"])
    replaced_words = []
    # Loop through each word in the sentence
    for word in dt:
      if word.upper() in df1['abb'].values:
            # Replace the word with the corresponding word from Word2 column of df1
            replaced_words.append(df1[df1['abb'] == word.upper()]['exp'].values[0])
      else:
            replaced_words.append(word)
    # Join the words back into a sentence
    replaced_text = ' '.join(replaced_words)
    # Update the Text column in df2 with the replaced text
    data.at[index, 'Text'] = replaced_text

In [15]:
nlp_bc = en_ner_bc5cdr_md.load()
nlp_bi = en_ner_bionlp13cg_md.load()

In [16]:
disease=[]
chemical=[]
for doc in nlp_bc.pipe(data['Text']):
  c=[]
  d=[]
  for x in doc.ents:
    ((x.text,x.label_))
    if x.label_=='DISEASE':
       d.append(x.text)
    elif x.label_=='CHEMICAL':
       c.append(x.text)
  if d!=[]:
    disease.append(list(set(d)))
  else:
    disease.append(None)
  if c!=[]:
    chemical.append(list(set(c)))
  else:
    chemical.append(None)
data['chemical']=chemical
data['disease']=disease

In [17]:
cm =  list(df1["chemical"][:475])

def get_matching_chemicals(chemicals):
    matching_chemicals = []
    for i in data['chemical']:
        if i is not None:
            matching_chemicals.append([d for d in i if d.capitalize() in cm])
        else:
            matching_chemicals.append(None)
    return matching_chemicals

data['Chemical']=get_matching_chemicals(data['chemical'])

In [18]:
df1['disease']=df1['disease'].str.capitalize()
dm =  list(df1["disease"][:1406])

def get_matching_diseases(diseases):
    matching_diseases = []
    for i in data['disease']:
        if i is not None:
            matching_diseases.append([d for d in i if d.capitalize() in dm])
        else:
            matching_diseases.append(None)
    return matching_diseases

data['Disease']=get_matching_diseases(data['disease'])

In [19]:
simple_chem=[]
for doc in nlp_bi.pipe(data['Text']):
  sc=[]
  for x in doc.ents:
    ((x.text,x.label_))
    if x.label_=='SIMPLE_CHEMICAL':
      sc.append(x.text)
  if sc!=[]:
    simple_chem.append(list(set(sc)))
  else:
    simple_chem.append(None)
data['simple_chem']=simple_chem

In [20]:
dm =  list(df1["simple_chemical"][:601])

def get_matching_simple_chemical(simple):
    matching_diseases = []
    for i in data['simple_chem']:
        if i is not None:
            matching_diseases.append([d for d in i if d.capitalize() in dm])
        else:
            matching_diseases.append(None)
    return matching_diseases

data['Simple_chem']=get_matching_simple_chemical(data['simple_chem'])

In [21]:
vaccines=['tricat','metacam','loxicom','ducat','nobivac','canigen','dhp','lepto4','l4','l2','flv','kcv','nsaid','kc','rv','bb','felv','fiv','rheumocam','gabapentin']
for index, row in data.iterrows():
    dt=word_tokenize(row['text'])
    replaced_words = []
    # Loop through each word in the sentence
    for word in dt:
      if word.lower() in vaccines:
        if word.lower() not in replaced_words:
            # Replace the word with the corresponding word from Word2 column of df1
            replaced_words.append(word.lower())
    replaced_text = ' '.join(replaced_words)
    # Update the Text column in df2 with the replaced text
    data.at[index, 'vacc'] = replaced_text

data['vac']=data['vacc']
data['vac'] = data['vac'].apply(lambda x: None if not x else x)
data['vac'] = data['vac'].str.replace(' ', ',')
data['vac'] = data['vac'].apply(lambda x: x.split(', ') if x is not None else None)

In [22]:
data['Chemical'] = data['Chemical'].apply(lambda x: None if x == [] else x)
data['vac'] = data['vac'].apply(lambda x: None if x == [] else x)
data['simple_chem'] = data['simple_chem'].apply(lambda x: None if x == [] else x)
data[['vac','Chemical','simple_chem']]

,vac,Chemical,simple_chem
3619,None,None,None
3417,[l4],[Nobivaclepto4],None
4328,None,None,None
523,[l4],None,None
1480,None,None,None
...,...,...,...
1674,None,None,None
3138,None,"[glutamate-oxaloacetate, Lethargy/inappetence,...","[brightener Intranasal, Flea/worm]"
2161,"[kc,dhp]",[nobivacdhp],None
768,"[loxicom,gabapentin]",[Dioctyl sodium sulfosuccinate],[q8-12h]


In [23]:
vacc_dict = {
'tricat': 'nobivac tricat',
'metacam': 'Metacam Meloxicam',
'loxicom': 'Loxicom Meloxicam',
'ducat': 'nobivac ducat',
'nobivac': 'nobivac',
'canigen': 'Canigen',
'dhp': 'nobivacdhp',
'lepto4': 'nobivaclepto4',
'l4': 'nobivaclepto4',
'l2': 'nobivaclepto2',
'flv': 'Feline Leukemia Virus Vaccine',
'kcv': 'Kennel Cough Vaccine',
'nsaid': 'Non-Steroidal Anti-Inflammatory Drug',
'kc': 'Kennel Cough vaccine',
'rv': 'Rabies Vaccine',
'bb': 'Bordetella Bronchiseptica Vaccine',
'felv': 'Feline Leukemia Virus Vaccine',
'fiv': 'Feline Immunodeficiency Virus Vaccine',
'rheumocam':'rheumocam',
'gabapentin':'gabapentin'
}

new_vacc=[]
for i in data['vac']:
  if i:
    for j in i:
      new_j=[]
      for x in word_tokenize(j):
        if x in vacc_dict:
          new_j.append(vacc_dict[x].capitalize())
      new_vacc.append(new_j)
  else:
      new_vacc.append(None)
data['vaccines']=new_vacc

In [24]:
merged_col = []
for i, row in data.iterrows():
  if row['Chemical'] is None and row['Simple_chem'] is None:
    merged_col.append(None)
  elif row['Chemical'] is None and row['Simple_chem'] is not None:
    merged_col.append(list(set(row['Simple_chem'])))
  elif row['Chemical'] is not None and row['Simple_chem'] is None:
    merged_col.append(list(set(row['Chemical'])))
  else:
    merged_col.append(list(set(row['Chemical']+row['Simple_chem'])))
data['All_Chemicals'] = merged_col

In [25]:
data_new=data[['SAVSNET_consult_id', 'SAVSNET MPC','Consult_date','Species','All_Chemicals','Chemical','chemical','Disease', 'disease', 'vaccines', 'vacc','simple_chem','Simple_chem','year','month']]
data_new.to_csv('final_data.csv',encoding='utf-8')

# **CHALLENGES**

In [26]:
disease_list = [d for d in data.disease if d is not None]
from collections import Counter

flat_lst = [item.capitalize() for sublist in disease_list for item in sublist]
word_count = Counter(flat_lst)
word_count = dict(pd.Series(flat_lst).value_counts())
df = pd.DataFrame({'word': list(word_count.keys()), 'count': list(word_count.values())})


disease_list = [d for d in data.disease if d is not None]
unique_diseases = np.unique(np.hstack(disease_list)).tolist()

chemical_list = [d for d in data.chemical if d is not None]
unique_chemicals=np.unique(np.hstack(chemical_list)).tolist()

simple_chem_list = [d for d in data.simple_chem if d is not None]
unique_simple_chemicals=np.unique(np.hstack(simple_chem_list)).tolist()

In [27]:
!pip install symspellpy

import pkg_resources
from symspellpy import SymSpell, Verbosity

# Load dictionary
dictionary_path = pkg_resources.resource_filename("symspellpy", "frequency_dictionary_en_82_765.txt")
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

# Define input text
input_text = "Boosetr: h c satis. O no concerns. Ce NAD. Unable to chek teeth due to muzle but O not concerned himself, no bad breth and what he has seen they are good to him. UTD worming and fleas treatment."

# Split text into words
words = input_text.split()

# Correct misspelled words
corrected_words = []
for word in words:
    suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2)
    corrected_word = suggestions[0].term if suggestions else word
    corrected_words.append(corrected_word)

# Join corrected words into a single string
output_text = " ".join(corrected_words)

print("Input text:", input_text)
print("Output text:", output_text)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 7.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for editdistpy: filename=editdistpy-0.1.3-cp310-cp310-linux_x86_64.whl size=141755 sha256=4a9db72f66afad3d09c32aaf4f56e5a6041e1e6dfcf75e8f578e2f5476e8f0b6
  Stored in directory: /root/.cache/pip/wheels/88/6a/a6/a1283cc145323a1fb3d475bd158ee60b248ab1985230d266fc
Successfully built editdistpy
Input text: Boosetr: h c satis. O no concerns. Ce NAD. Unable to chek teeth due to muzle but O not concerned himself, no bad breth and what he has seen they are good to him. UTD worming and fleas treatment.
Output text: Boosetr: a a satisfy a no concerns be NAD. enable to check teeth due to mule bu

In [32]:
input_text="lame RF 5 days possibly getting better - had left over melox at weekend but usually causes diarr - CE NAD advise regarding likely STI - continue rest/wt loss & use different nsaid."
words = input_text.split()

# Correct misspelled words
corrected_words = []
for word in words:
    suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2)
    corrected_word = suggestions[0].term if suggestions else word
    corrected_words.append(corrected_word)

# Join corrected words into a single string
output_text = " ".join(corrected_words)

print("Input text:", input_text)
print("Output text:", output_text)

Input text: lame RF 5 days possibly getting better - had left over melox at weekend but usually causes diarr - CE NAD advise regarding likely STI - continue rest/wt loss & use different nsaid.
Output text: lame of a days possibly getting better a had left over melon at weekend but usually causes diary a of NAD advise regarding likely STI a continue restart loss a use different said


In [28]:
#!pip install contextualSpellCheck
import contextualSpellCheck
import spacy
nlp = spacy.load("en_ner_bionlp13cg_md") 

contextualSpellCheck.add_to_pipe(nlp)
nlp.pipe_names

doc = nlp('nobivacdhp and Nobivaclepto4 and Kennel Cough . Bright Alert Responsive dude No abnormality detected / No apparent distress .')
doc._.outcome_spellCheck

'and and Nobivaclepto4 and the Cough. Bright Alerts dude No alarm detected / No apparent distress.'